In [1]:
import proteusAI as pai
import pandas as pd

ImportError: cannot import name 'filter_backbone' from 'biotite.structure' (/opt/anaconda3/lib/python3.11/site-packages/biotite/structure/__init__.py)

In [ ]:
library_pae = pai.Library(source='demo/demo_data/master_dataset.csv', seqs_col='binder_seq', y_col='pae_interaction_HLA-A101-RVTDESILSY', 
                    y_type='num', names_col='binder_name')
library_plddt = pai.Library(source='demo/demo_data/master_dataset.csv', seqs_col='binder_seq', y_col='plddt_binder_HLA-A101-RVTDESILSY', 
                    y_type='num', names_col='binder_name')

In [ ]:
model_pae = pai.Model(library=library_pae)
plddt_model = pai.Model(library=library_plddt)

In [ ]:
_ = model_pae.train(k_folds=5, model_type='rf', x='blosum62', seed=42, split=(60,20,20))

In [ ]:
model_pae.true_vs_predicted(y_true=model_pae.y_val, y_pred=model_pae.y_val_pred)

In [ ]:
train_names = [prot.name for prot in model_pae.train_data]
test_names = [prot.name for prot in model_pae.test_data]
val_names = [prot.name for prot in model_pae.val_data]

In [ ]:
train = [prot for prot in plddt_model.library.proteins if prot.name in train_names]
test = [prot for prot in plddt_model.library.proteins if prot.name in test_names]
val = [prot for prot in plddt_model.library.proteins if prot.name in val_names]

split = {'train':train, 'test':test, 'val':val}

plddt_out = plddt_model.train(k_folds=5, model_type='rf', x='blosum62', seed=42, split=split)

In [ ]:
plddt_model.true_vs_predicted(y_true=plddt_model.y_val, y_pred=plddt_model.y_val_pred)

In [ ]:
pae_out = model_pae.search(optim_problem='min', overwrite=True) # acq_fn = 'ei'

In [ ]:
predicted_proteins = [pai.Protein(seq=row.sequence, name=row['name']) for i, row in pae_out.iterrows()]
plddt_predictions = plddt_model.predict(predicted_proteins)

In [ ]:
for prot in plddt_predictions[0]:
    print(prot)
    predicted_plddt_value = prot.y_pred
    predicted_plddt_sigma = prot.y_sigma
    print(predicted_plddt_value, predicted_plddt_sigma)
    break

In [ ]:
out = out.sort_values(by=['y_predicted'], ascending=True)
out.to_csv('demo/demo_data/predictions.csv')

In [ ]:
out.hist()